In [1]:
# coding: utf-8
"""
提案手法を実験するためのコード
"""
from filer2.filer2 import Filer
from library.submodular import GraphSubModular, Rouge_N
import numpy as np
import glob

In [4]:
list_test_path = glob.glob('./files/OpinosisDataset1.0/preprocessed/test/*')
list_ans_path = glob.glob('./files/OpinosisDataset1.0/preprocessed/ans/*')

list_test_path.sort()
list_ans_path.sort()

In [ ]:
directed = False

for test_path, ans_path in zip(list_test_path, list_ans_path):
    dict_test = Filer.readdump(test_path)
    dict_ans = Filer.readdump(ans_path)
    filename = ans_path.replace('./files/OpinosisDataset1.0/preprocessed/ans/', '')
    filename = filename.replace('.dump', '')
    # テスト用ファイルの作成
    # １００単語以上ある文は除く
    list_sentence = []
    list_sep = []
    list_sep_all = []
    list_edgelist = []
    for sen, sep, sepall, edge in zip(dict_test['sentence'],
                                      dict_test['sep_njv'],
                                      dict_test['sep_all'],
                                      dict_test['edge_njv']):
        if len(sepall) <= 100:
            list_sentence.append(sen)
            list_sep.append(sep)
            list_sep_all.append(sepall)
            list_edgelist.append(edge)

    # エッジリストの方向を変えることができる
    list_edgelist = [[row1[0], row1[1]] for row in list_edgelist for row1 in row]
    
    # インスタンス化
    GM = GraphSubModular(list_sep_all=list_sep_all,
                         list_sep=list_sep,
                         list_edgelist=None,
                         directed=directed,
                         inverse_flag=True)

    for scale in [0,1,2]:
        for r in [0.1, 0.5, 1, 2]:
            # 学習
            GM.m_greedy(num_w=100, r=r, scale=scale)
            # Rougeを計算するようにbag_of_wordsを作成する
            list_gm_test = [row[2] for row in GM.list_C]
            for row in dict_ans['sep_all']:
                R1 = Rouge_N.rouge(list_gm_test, row, N=1)
                R2 = Rouge_N.rouge(list_gm_test, row, N=2)
                R3 = Rouge_N.rouge(list_gm_test, row, N=3)
                R4 = Rouge_N.rouge(list_gm_test, row, N=4)
                list_result = [[filename, scale, r, R1, R2, R3, R4]]
                Filer.writecsv(list_result, './files/OpinosisDataset1.0/result/GSM/njv/njv_cor_%s_%s.csv'%(scale, r))

計算が終了しました
property: list_C
計算が終了しました
property: list_C
計算が終了しました